In [0]:
session.sql('use weather')


Out[25]: DataFrame[]

In [0]:
%run /Users/tha075bei026@tcioe.edu.np/actionLogger

In [0]:
from pyspark.sql.functions import *
from datetime import datetime
import uuid

def load_cleaned_data():
    
    process_start_time = session.sql("SELECT current_timestamp()").collect()[0][0]
    log_dict = {'id': str(uuid.uuid4().hex),
            'load_type': 'processed',
            'table_name': 'raw_weather_table',
            
            'process_start_time':  session.sql("SELECT current_timestamp()").collect()[0][0],
            'process_end_time': session.sql("SELECT current_timestamp()").collect()[0][0],

            'start_date_time':  session.sql("SELECT current_timestamp()").collect()[0][0],
            'end_date_time':  session.sql("SELECT current_timestamp()").collect()[0][0]}
    
    
    log = action_logger(log_dict)
    
    try:
        dfc = session.read.format('delta').load('dbfs:/databases/weather/raw_weather_table')


        dfr = dfc.select(col("dt"),
                 col("id").alias("city_id"),
                 col("name").alias("city_name"),
                 col("timezone"),
                 col("country"),
                 col("lat"),
                 col("lon"),
                 col("temp"),
                 col("temp_min"),
                 col("temp_max"),
                 col("pressure"),
                 col("humidity"),
                 col("visibility"),
                 col("wind_speed"),
                 col("wind_deg"),
                 col("wind_gust"),
                 col("all").alias("all_1h"),
                 col("description").alias("clouds"),
                 col("load_run_id"),
                 col("created_on"),
                 col("created_by")
                )
        dfr = dfr.withColumn("datetime", date_format(from_unixtime(col("dt")), "yyyy-MM-dd HH:mm:ss"))
        
        process_end_time = session.sql("SELECT current_timestamp()").collect()[0][0]
        
        log_dict = {
               'process_start_time' : process_start_time,
               'process_end_time' : process_end_time,
               'status' : 'extracting',
               'start_date_time' : dfr.select(min('created_on')).first()[0],
               'end_date_time' : dfr.select(max('created_on')).first()[0]}
        
        log.action(log_dict)
        
        dfr = dfr.distinct()
    
#         dfr.write.format('delta').mode('append').save('dbfs:/databases/weather/weather_table_cleaned')
        dfr.write.option("overwriteSchema", "true")\
                .format('delta')\
                .mode('append')\
                .save('dbfs:/databases/weather/cleaned_weather_table')
        
        process_end_time = session.sql("SELECT current_timestamp()").collect()[0][0]
        log_dict = {
               'process_start_time' : process_start_time,
               'process_end_time' : process_end_time,
               'status' : 'completed',
               'start_date_time' : dfr.select(min('created_on')).first()[0],
               'end_date_time' : dfr.select(max('created_on')).first()[0]}
        
        log.action(log_dict)
        
    except Exception as e:
        error_dict = {
               'process_start_time' : process_start_time,
               'process_end_time' : session.sql("SELECT current_timestamp()").collect()[0][0],
               'status' : 'error',
               'error_data' : e,
               'start_date_time' : session.sql("SELECT current_timestamp()").collect()[0][0],
               'end_date_time' : session.sql("SELECT current_timestamp()").collect()[0][0]
        }
        log.action(error_dict)
    